In [ ]:
import pickle

import vps_tester
from hetznercloud import constants

### Save keys

In [ ]:
hetzner_api_key = ''
ssh_file_name = ''

with open('secret.pkl', 'rb') as f:
    pickle.dump({'hetzner_api_key': hetzner_api_key, 'ssh_file_name': ssh_file_name}, f)

### Load keys

In [ ]:
with open('secret.pkl', 'rb') as f:
    secret = pickle.load(f)
    
ssh_file_name = secret['ssh_file_name']
hetzner_api_key = secret['hetzner_api_key']

### Test single instance performance

In [ ]:
h = vps_tester.HetznerTester(hetzner_api_key, ssh_file_name)
h.test_vps(constants.DATACENTER_HELSINKI_1, constants.SERVER_TYPE_1CPU_2GB)

### Compare different instance types

In [ ]:
for server_type in [constants.SERVER_TYPE_1CPU_2GB,
                   constants.SERVER_TYPE_2CPU_8GB,
                   constants.SERVER_TYPE_2CPU_8GB_DVCPU,
                  ]:
    result, _ = h.test_vps(constants.DATACENTER_HELSINKI_1, server_type)

    print(f'server type: {server_type}, result: {result}')

### Copmare different datacenters

In [ ]:
for datacenter in [constants.DATACENTER_FALKENSTEIN_1,
                   constants.DATACENTER_HELSINKI_1,
                   constants.DATACENTER_NUREMBERG_1,
                  ]:
    result, _ = h.test_vps(datacenter, constants.SERVER_TYPE_1CPU_2GB)

    print(f'datacenter: {datacenter}, result: {result}')

### Find best instance in any datacenter, shutdown others

In [ ]:
servers = []
results = []

for datacenter in [constants.DATACENTER_FALKENSTEIN_1,
                   constants.DATACENTER_HELSINKI_1,
                   constants.DATACENTER_NUREMBERG_1,
                  ]:
    result, server = h.test_vps(datacenter, 
                                constants.SERVER_TYPE_1CPU_2GB,
                                kill_instantly=False)
    
    results.append(int(result))
    servers.append(server)
    print(f'datacenter: {datacenter}, result: {result}')
    
    best_server = servers[results.index[min[results]]]
    for server in [s in servers if s is not best_server]:
        h.delete_server(server)

print(f'best one: {best_server.public_net_ipv4}')